In [ ]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import gc

In [ ]:
X_dict = dict()
for root, dirs, files in os.walk("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/lamono_tesnors"):
    for filename in files:
      sparse_tensor = torch.load("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/lamono_tesnors/"+filename)
      X_dict[filename[:-3]] = sparse_tensor

In [ ]:
A_hat = torch.load("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/A_hat.pt")

In [ ]:
class GCNLayer(nn.Module):
    """
        GCN layer

        Args:
            input_dim (int): Dimension of the input
            output_dim (int): Dimension of the output (a softmax distribution)
            A (torch.Tensor): 2D adjacency matrix
    """

    def __init__(self,  input_dim, output_dim, A: torch.Tensor,activation = F.relu):
        super(GCNLayer, self).__init__()
        A = A.coalesce()
        self.activation = activation

        #(D^-1/2 * A_hat * D^-1/2)
        #each element of A_hat aij should be multiplied on 1/(di*dj)^(1/2)
        #where di - number of graph edges of i node
        #dj - number of graph edges of j node
        #Since we have removed the edge pixels , each vertex will have 8 neighbours.
        #So each element of A_hat matrix should be multiplied on 1/(8*8)^(1/2) = 1/8

        A_fin = torch.sparse_coo_tensor(
            A.indices(),
            A.values() * 1/8,
            A.size()
        )
        self.A_fin = A_fin
        self.W = nn.Parameter(torch.rand(input_dim, output_dim)) #glorot_init(input_dim, output_dim)

    def forward(self, X: torch.Tensor):

        # (D^-1/2 * A_hat * D^-1/2) * X
        support_1 = torch.matmul(X.reshape(-1,1), self.W)

        # (D^-1/2 * A_hat * D^-1/2) * X * W
        support_2 = torch.sparse.mm(self.A_fin, support_1)
        # ReLU(D^-1/2 * A_hat * D^-1/2 * X * W)
        outputs = self.activation(support_2)
        return outputs

In [ ]:
# def dot_product_decode(Z):
#   z_sp = Z.to_sparse()
#   zt_sp = Z.t().to_sparse()
#   A_pred = torch.sparse.mm(z_sp,Z.t())
#   return A_pred

In [ ]:
def dot_product_decode(Z, batch_size=1000):
    N = Z.size(0)
    A_pred = torch.zeros(N, N)  # Ініціалізація результату

    for i in range(0, N, batch_size):
        Z_i = Z[i:i+batch_size]  # Вибір блоку рядків
        A_pred[i:i+batch_size] = torch.matmul(Z_i, Z.T)

    return A_pred

In [ ]:
class VGAE(nn.Module):
  def __init__(self, input_dim,hidden1_dim,hidden2_dim,A):
    super(VGAE, self).__init__()
    self.input_dim = input_dim
    self.hidden1_dim = hidden1_dim
    self.hidden2_dim = hidden2_dim
    self.base_gcn = GCNLayer(input_dim, hidden1_dim, A)
    self.gcn_mean = GCNLayer(hidden1_dim, hidden2_dim, A, activation=lambda x:x)
    self.gcn_logstddev = GCNLayer(hidden1_dim, hidden2_dim, A, activation=lambda x:x)

  def encode(self, X):
    hidden = self.base_gcn(X)
    self.mean = self.gcn_mean(hidden)
    self.logstd = self.gcn_logstddev(hidden)
    gaussian_noise = torch.randn(X.size(0), self.hidden2_dim)
    sampled_z = gaussian_noise*torch.exp(self.logstd) + self.mean
    return sampled_z

  def forward(self, X):
    Z = self.encode(X)
    print(Z.t())
    A_pred = dot_product_decode(Z)
    return A_pred

In [ ]:
# Create the GCN Layer
VGAE_model = VGAE(1,1,1,A_hat)

# Example input feature matrix
X = X_dict['001'].to_dense().to(torch.float)

output = VGAE_model.forward(X)

print(output)